<a href="https://colab.research.google.com/github/Harlequelrah/Learning_data_analysis/blob/main/Learning_data_analysis/Courses/Partie_2/exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercice pour nettoyer les données de operations.csv

In [37]:
import pandas as pd
import numpy as np

# 1 - Importation des données et verification des 5 premières lignes

In [ ]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/Harlequelrah/Learning_data_analysis/refs/heads/main/Learning_data_analysis/Nettoyez_et_analysez_votre_jeu_de_données/data/operations.csv"
)
data.head()

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE


# 2 - Verifions les erreurs de types

In [39]:
display(data.dtypes)

,0
date_operation,object
libelle,object
montant,float64
solde_avt_ope,float64
categ,object


In [40]:
data['date_operation']=pd.to_datetime(data['date_operation'])

# 3 - Verifions les valeurs manquantes

In [41]:
nb_na=data.isnull().sum()
nb_na[nb_na>0] # les valeurs manquantes

,0
montant,2
categ,1


In [42]:
data_na=data.loc[data['montant'].isnull(),:] # trouver les individus pour lesquels le montant est null

In [43]:
# Iterate through the indices of the original data DataFrame where 'montant' is null
for index in data.loc[data['montant'].isnull()].index:
  # Calculate the 'montant' using the difference between the next row's 'solde_avt_ope'
  # and the current row's 'solde_avt_ope' from the original data DataFrame
  data.loc[index,'montant'] = data.loc[index+1,'solde_avt_ope'] - data.loc[index,'solde_avt_ope']

# The current amount would be the difference of the balance after the operation of the next operation,
# which represents its amount after the operation, and the balance before the operation of the current operation.

In [44]:
data.loc[data['categ'].isnull(),:] # recherchons l'individu avec la catégorie nulle

,date_operation,libelle,montant,solde_avt_ope,categ
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN


In [45]:
data.loc[data['libelle']=='PRELEVEMENT XX TELEPHONE XX XX',:]
# cherchons à partir des autres informations ici en l'occurence le libelle si on ne peut pas obtenir la catégorie

,date_operation,libelle,montant,solde_avt_ope,categ
8,2023-04-05,PRELEVEMENT XX TELEPHONE XX XX,-7.02,2056.02,FACTURE TELEPHONE
62,2023-05-09,PRELEVEMENT XX TELEPHONE XX XX,-7.02,4090.10,FACTURE TELEPHONE
102,2023-06-07,PRELEVEMENT XX TELEPHONE XX XX,-6.38,4688.91,FACTURE TELEPHONE
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN
204,2023-08-07,PRELEVEMENT XX TELEPHONE XX XX,-7.46,3751.73,FACTURE TELEPHONE
260,2023-09-05,PRELEVEMENT XX TELEPHONE XX XX,-6.38,3453.96,FACTURE TELEPHONE
308,2023-10-06,PRELEVEMENT XX TELEPHONE XX XX,-13.58,2413.58,FACTURE TELEPHONE


In [46]:
data.loc[data['categ'].isnull(),'categ'] = 'PRELEVEMENT XX TELEPHONE XX XX'

# 3 - Examinons la présence de doublon

In [49]:
columns = data.columns.tolist()
data.loc[data[columns].duplicated(keep=False)] # recherche de doublons

,date_operation,libelle,montant,solde_avt_ope,categ
43,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES
44,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES


In [50]:
data.drop_duplicates(subset=columns,inplace=True,ignore_index=True)

In [ ]:
# 4 - Detections d'outliers

In [51]:
data.describe()

,date_operation,montant,solde_avt_ope
count,308,308.000000,308.000000
mean,2023-07-05 10:59:13.246753280,-45.782013,3395.301071
min,2023-03-31 00:00:00,-15000.000000,1416.810000
25%,2023-05-21 06:00:00,-20.447500,3010.737500
50%,2023-07-05 12:00:00,-9.600000,3452.465000
75%,2023-08-21 00:00:00,-2.715000,3787.232500
max,2023-10-06 00:00:00,1071.600000,4709.310000
std,NaN,872.818105,667.109412


Le montant minimum est -150.000 mais aucun solde avant ne semble l'expliquer .

In [53]:
i = data.loc[data['montant']==-15000,:].index[0] # index de l'individu avec le montant -150000
data.iloc[i-1:i+2,:]

,date_operation,libelle,montant,solde_avt_ope,categ
197,2023-08-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,3121.35,AUTRE
198,2023-08-03,CARTE XX XX XX XX,-15000.00,3797.35,AUTRE
199,2023-08-03,CARTE XX XX L'EPICERIE DEMBAS XX XX,-10.51,3782.96,AUTRE


In [54]:
data.loc[data['montant']==-15000,'montant'] = -14.39 # -14.39 est le vrai montant de l'operation en faisant la différence du solde_avt_ope du prochain - le solde_avt_ope actuelle